In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import nlppln

wf = nlppln.WorkflowGenerator()
wf.load(steps_dir='../cwl/')
print(wf.list_steps())

in_dir = wf.add_inputs(in_dir='Directory')

folia_files = wf.ls(in_dir=in_dir)
result = wf.folia2ocr_and_gs(in_file=folia_files, scatter='in_file', scatter_method='flat_crossproduct')

wf.add_outputs(texts=result)
wf.save('../cwl/folia2texts.cwl')

Steps
  chunk-list-of-files...... file_list = wf.chunk_list_of_files(chunk_size, in_files)
  ixa-pipe-tok............. out_file = wf.ixa_pipe_tok(language, in_file)
  frog-single-text......... frogout = wf.frog_single_text(in_file)
  save-ner-data............ ner_statistics = wf.save_ner_data(in_files)
  download................. out_files = wf.download(urls)
  select-folia-files....... out_files = wf.select_folia_files(in_dir)
  rename-and-copy-files.... out_files = wf.rename_and_copy_files(in_files)
  folia2ocr-and-gs......... out_files = wf.folia2ocr_and_gs(in_file[, out_dir])
  apachetika............... out_files = wf.apachetika(in_files[, tika_server])
  ls....................... out_files = wf.ls(in_dir[, recursive])
  xml-to-text.............. out_files = wf.xml_to_text(in_files[, tag])
  frog-dir................. frogout = wf.frog_dir(dir_in[, skip])
  frog-to-saf.............. saf = wf.frog_to_saf(in_files)
  saf-to-freqs............. freqs = wf.saf_to_freqs(in_files[, mode])
